In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Library

In [ ]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.6/599.6 kB 19.6 MB/s eta 0:00:00


In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
from sklearn.model_selection import train_test_split
import cv2
import torch
from IPython.display import Image, clear_output  # to display images

from os import listdir
from os.path import isfile, join
import yaml

In [ ]:
import os
def count_file(folder_path):
    # Get the list of files in the folder
    file_list = os.listdir(folder_path)

    # Count the number of files
    num_files = len(file_list)

    # Print the number of files
    print(f"Folder '{folder_path}' : {num_files} files.")

# Git Yolo v5

In [ ]:
%cd /content

/content


In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 15935, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 15935 (delta 67), reused 74 (delta 52), pack-reused 15831
Receiving objects: 100% (15935/15935), 14.59 MiB | 8.08 MiB/s, done.
Resolving deltas: 100% (10940/10940), done.


# **Root** ( change this for each process)

ĐỔI path_of_root thành path của nơi để folder gốc

In [ ]:
# Nơi để folder
path_of_root="/content/drive/MyDrive/"

#data png unzip
vinbigdata="/content/Normal/"

#Working for Normal
working = path_of_root + "DS312 - Xử lí ảnh y khoa/Project/Working/Normal_V5/"

In [ ]:
path_of_data=working + 'data_yaml/vinbigdata.yaml'

# Unzip Data

In [ ]:
%cd /content

/content


In [ ]:
zip = path_of_root + "DS312 - Xử lí ảnh y khoa/Project/VINBIGDATA/Data/Normal.zip"
!unzip "{zip}"

In [ ]:
count_file("/content/Normal")

Folder '/content/Normal' : 15000 files.


# Import Data

train.csv

In [ ]:
train_df = pd.read_csv(path_of_root + 'DS312 - Xử lí ảnh y khoa/Project/VINBIGDATA/Data/train.csv')
train_df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height
0,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0,2080,2336
1,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0,2304,2880
2,1c32170b4af4ce1a3030eb8167753b06,Pleural thickening,11,R9,627.0,357.0,947.0,433.0,2540,3072
3,0c7a38f293d5f5e4846aa4ca6db4daf1,ILD,5,R17,1347.0,245.0,2188.0,2169.0,2285,2555
4,47ed17dcb2cbeec15182ed335a8b5a9e,Nodule/Mass,8,R9,557.0,2352.0,675.0,2484.0,2568,3353


In [ ]:
len(train_df['image_id'].unique())

3515

val.csv

In [ ]:
val_df = pd.read_csv(path_of_root +'DS312 - Xử lí ảnh y khoa/Project/VINBIGDATA/Data/val.csv')
len(val_df['image_id'].unique())

439

test.csv

In [ ]:
test_df = pd.read_csv(path_of_root +'DS312 - Xử lí ảnh y khoa/Project/VINBIGDATA/Data/test.csv')
len(test_df['image_id'].unique())

440

read image file

In [ ]:
train_df['image_path'] = vinbigdata+train_df.image_id+'.png'
train_df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,image_path
0,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0,2080,2336,/content/Normal/9a5094b2563a1ef3ff50dc5c7ff713...
1,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0,2304,2880,/content/Normal/051132a778e61a86eb147c7c6f564d...
2,1c32170b4af4ce1a3030eb8167753b06,Pleural thickening,11,R9,627.0,357.0,947.0,433.0,2540,3072,/content/Normal/1c32170b4af4ce1a3030eb8167753b...
3,0c7a38f293d5f5e4846aa4ca6db4daf1,ILD,5,R17,1347.0,245.0,2188.0,2169.0,2285,2555,/content/Normal/0c7a38f293d5f5e4846aa4ca6db4da...
4,47ed17dcb2cbeec15182ed335a8b5a9e,Nodule/Mass,8,R9,557.0,2352.0,675.0,2484.0,2568,3353,/content/Normal/47ed17dcb2cbeec15182ed335a8b5a...


In [ ]:
val_df['image_path'] = vinbigdata+val_df.image_id+'.png'
val_df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,image_path
0,d3637a1935a905b3c326af31389cb846,Aortic enlargement,0,R10,1329.0,743.0,1521.0,958.0,2304,2880,/content/Normal/d3637a1935a905b3c326af31389cb8...
1,0a14aeaf02d42990d8bb5d55270b7274,ILD,5,R9,593.0,1597.0,1111.0,2207.0,2774,3043,/content/Normal/0a14aeaf02d42990d8bb5d55270b72...
2,ebf4ffbcf6de699bf9deab70f0dee8ff,Cardiomegaly,3,R9,876.0,1810.0,2336.0,2610.0,2779,2912,/content/Normal/ebf4ffbcf6de699bf9deab70f0dee8...
3,eb6c714df22142229464c6b83e47d7d6,Infiltration,6,R8,439.0,1810.0,961.0,2452.0,2532,3073,/content/Normal/eb6c714df22142229464c6b83e47d7...
4,fa4c38b3c5e53e1e84bd0923f2d8e480,Aortic enlargement,0,R9,1397.0,849.0,1661.0,1200.0,2304,2880,/content/Normal/fa4c38b3c5e53e1e84bd0923f2d8e4...


In [ ]:
test_df['image_path'] = vinbigdata+test_df.image_id+'.png'
test_df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,image_path
0,18a61a07e6f5f13ebfee57fa36cd8b6f,Pulmonary fibrosis,13,R9,393.0,283.0,822.0,643.0,2304,2880,/content/Normal/18a61a07e6f5f13ebfee57fa36cd8b...
1,d7aea6f37597df566725da73655edcc2,Infiltration,6,R8,1768.0,1137.0,2346.0,2020.0,2594,3395,/content/Normal/d7aea6f37597df566725da73655edc...
2,0cfae3f740f87ca3b96dec765d06fad0,Aortic enlargement,0,R9,1142.0,738.0,1451.0,1119.0,2304,2880,/content/Normal/0cfae3f740f87ca3b96dec765d06fa...
3,b42e930c16c0166dbeae813b47bb8b07,Pleural thickening,11,R9,1972.0,401.0,2250.0,574.0,3072,3072,/content/Normal/b42e930c16c0166dbeae813b47bb8b...
4,d07557904cbe57fc1bdebac1e8aeefa1,Nodule/Mass,8,R9,378.0,899.0,403.0,929.0,2304,2880,/content/Normal/d07557904cbe57fc1bdebac1e8aeef...


In [ ]:
classes = train_df['class_name'].unique().tolist()
classes =sorted(classes)
classes

# Prepare DATA train, val, test

In [ ]:
train_files = train_df['image_path'].unique()
val_files   = val_df['image_path'].unique()
test_files  = test_df['image_path'].unique()
len(train_files), len(val_files), len(test_files)

(3515, 439, 440)

## Copying Files
- ĐÃ CÓ DATA TRONG FOLDER THÌ KHÔNG CẦN CHẠY

In [ ]:
count_file(path_of_root + 'DS312 - Xử lí ảnh y khoa/Project/VINBIGDATA/vinbigdata_label')

Folder '/content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/VINBIGDATA/vinbigdata_label' : 4394 files.


In [ ]:
os.makedirs(working+'vinbigdata/labels/train', exist_ok = True)
os.makedirs(working+'vinbigdata/labels/val', exist_ok = True)
os.makedirs(working+'vinbigdata/labels/test', exist_ok = True)


os.makedirs(working+'vinbigdata/images/train', exist_ok = True)
os.makedirs(working+'vinbigdata/images/val', exist_ok = True)
os.makedirs(working+'vinbigdata/images/test', exist_ok = True)

label_dir = path_of_root + 'DS312 - Xử lí ảnh y khoa/Project/VINBIGDATA/vinbigdata_label'

In [ ]:
for file in tqdm(train_files):
    shutil.copy(file, working+'vinbigdata/images/train')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), working+'vinbigdata/labels/train')

for file in tqdm(val_files):
    shutil.copy(file, working+'vinbigdata/images/val')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), working+'vinbigdata/labels/val')

for file in tqdm(test_files):
    shutil.copy(file, working+'vinbigdata/images/test')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), working+'vinbigdata/labels/test')

  0%|          | 0/3515 [00:00<?, ?it/s]

  0%|          | 0/439 [00:00<?, ?it/s]

  0%|          | 0/440 [00:00<?, ?it/s]

# YOLO v5 make .ymal and folder data


In [ ]:
os.makedirs(working+'data_yaml', exist_ok = True)

with open(join( working , 'data_yaml/train.txt'), 'w') as f:
    for path in glob(working+'vinbigdata/images/train/*'):
        f.write(path+'\n')

with open(join( working , 'data_yaml/val.txt'), 'w') as f:
    for path in glob(working+'vinbigdata/images/val/*'):
        f.write(path+'\n')

with open(join( working , 'data_yaml/test.txt'), 'w') as f:
    for path in glob(working+'vinbigdata/images/test/*'):
        f.write(path+'\n')


data = dict(
    test   =  join( working , 'data_yaml/test.txt' ),
    val   =  join( working , 'data_yaml/val.txt' ),
    train =  join( working , 'data_yaml/train.txt') ,
    nc    = 14,
    names = classes
    )

with open(join( working , 'data_yaml/vinbigdata.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(join( working , 'data_yaml/vinbigdata.yaml'), 'r')
print('\nyaml:')
print(f.read())


yaml:
names:
- Aortic enlargement
- Atelectasis
- Calcification
- Cardiomegaly
- Consolidation
- ILD
- Infiltration
- Lung Opacity
- Nodule/Mass
- Other lesion
- Pleural effusion
- Pleural thickening
- Pneumothorax
- Pulmonary fibrosis
nc: 14
test: "/content/drive/MyDrive/DS312 - Xu\u031B\u0309 li\u0301 a\u0309nh y khoa/Project/Working/Normal_V5/data_yaml/test.txt"
train: "/content/drive/MyDrive/DS312 - Xu\u031B\u0309 li\u0301 a\u0309nh y khoa/Project/Working/Normal_V5/data_yaml/train.txt"
val: "/content/drive/MyDrive/DS312 - Xu\u031B\u0309 li\u0301 a\u0309nh y khoa/Project/Working/Normal_V5/data_yaml/val.txt"



# Training

CHECK FILE BEFORE TRAIN:
- Train : 3515
- Val : 439
- Test : 440

if not enough files : reconnect the Drive until enough

In [ ]:
count_file(working+'vinbigdata/images/train'), count_file(working+'vinbigdata/labels/train')

Folder '/content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Normal_V5/vinbigdata/images/train' : 3515 files.
Folder '/content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Normal_V5/vinbigdata/labels/train' : 3515 files.


(None, None)

In [ ]:
count_file(working+'vinbigdata/images/val'), count_file(working+'vinbigdata/labels/val')

Folder '/content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Normal_V5/vinbigdata/images/val' : 439 files.
Folder '/content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Normal_V5/vinbigdata/labels/val' : 439 files.


(None, None)

In [ ]:
count_file(working+'vinbigdata/images/test'), count_file(working+'vinbigdata/labels/test')

Folder '/content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Normal_V5/vinbigdata/images/test' : 440 files.
Folder '/content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Normal_V5/vinbigdata/labels/test' : 440 files.


(None, None)

TRAIN

In [ ]:
os.makedirs(working+'YOLO', exist_ok = True)

os.chdir(working+'YOLO')

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 2.0.1+cu118 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15101MB, multi_processor_count=40)


In [ ]:
count_file(working+'vinbigdata/images/train'), count_file(working+'vinbigdata/labels/train')

Folder '/content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Normal_V5/vinbigdata/images/train' : 3515 files.
Folder '/content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Normal_V5/vinbigdata/labels/train' : 3515 files.


(None, None)

In [ ]:
!pwd

/content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Normal_V5/YOLO


Change path : data_path

In [ ]:
py_file_train = r"/content/yolov5/train.py"

In [ ]:
!python $py_file_train --img 640 --batch 16 --epochs 20 --data "$path_of_data" --weights yolov5l.pt

requirements: Ultralytics requirement "gitpython" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 301.7 MB/s eta 0:00:00

requirements: 1 package updated per ['gitpython']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

train: weights=yolov5l.pt, cfg=, data=/content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Normal_V5/data_yaml/vinbigdata.yaml, hyp=../../../../../../../yolov5/data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=../../../../../../../yolov5/runs/train, na

In [ ]:
import shutil


os.makedirs(working+'YOLO/runs', exist_ok = True)


source_folder = "/content/yolov5/runs"
destination_folder = working+"YOLO/runs"

# Remove the destination_folder if it already exists
if os.path.exists(destination_folder):
    shutil.rmtree(destination_folder)

# Copy the source_folder to the destination_folder
shutil.copytree(source_folder, destination_folder)

'/content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Normal_V5/YOLO/runs'

# path runs of train, predict

# path runs of train, predict
- Sau khi training thì sẽ lưu vào path "../../../../../../../yolov5/runs/train/exp"
- Chọn train_path sau khi train ( vd : exp="exp" )

In [ ]:
import os

folder_path = working+"YOLO/runs/train/"
folders = os.listdir(folder_path)

train_folders = [folder for folder in folders if folder.startswith("exp")]
train_folders.sort(reverse=True)  # Sắp xếp giảm dần theo thứ tự từ điển

if len(train_folders) > 0:
    latest_train_folder = train_folders[0]
    print(latest_train_folder)
else:
    print("Không tìm thấy folder bắt đầu bằng 'exp'.")

exp=latest_train_folder

exp


# EVALUATE

In [ ]:
%cd /content

/content


In [ ]:
path_of_model_train= working+ f"YOLO/runs/train/{exp}/weights/best.pt"
path_of_val_py= path_of_root +"/DS312 - Xử lí ảnh y khoa/Project/Working/yolov5/val.py"

## Valid


In [ ]:
!python "$path_of_val_py" --weights "$path_of_model_train" --data "$path_of_data" --img 640

val: data=/content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Normal_V5/data_yaml/vinbigdata.yaml, weights=['/content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Normal_V5/YOLO/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
unknown option: -
usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           <command> [<args>]
YOLOv5 🚀 2023-5-24 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4,

## Test

In [ ]:
!python "$path_of_val_py" --weights "$path_of_model_train" --data "$path_of_data" --img 640 --task test

val: data=/content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Normal_V5/data_yaml/vinbigdata.yaml, weights=['/content/drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/Normal_V5/YOLO/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=drive/MyDrive/DS312 - Xử lí ảnh y khoa/Project/Working/yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
unknown option: -
usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           <command> [<args>]
YOLOv5 🚀 2023-5-24 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4